In [ ]:
import torch
from lib.utils import load_raw_list,load_psd,load_raw
from torch.nn.functional import one_hot
idx = 0

In [ ]:
## Run this cell with range from 0 to 10, 10 to 20, 20 to 30, 30 to 32
import pandas as pd
import numpy as np
ret = pd.DataFrame()
idxs = list(range(30,32))
raw = load_raw(idxs[0])
df = load_psd(idxs[0])
eeg = raw.get_data(picks='EEG')[0]
X = pd.DataFrame(eeg.reshape(-1,5000))
y = df['label']

append = pd.concat([y,X],axis=1)
ret = pd.concat([ret,append])
for i in idxs[1:]:
    raw = load_raw(i)
    df = load_psd(i)
    eeg = raw.get_data(picks='EEG')[0]
    X = pd.DataFrame(eeg.reshape(-1,5000))
    y = df['label']

    append = pd.concat([y,X],axis=1)
    ret = pd.concat([ret,append])

ret = ret.reset_index(drop=True)
ret = ret[ret['label'] != 'X']

y = np.array(pd.Categorical(ret.pop('label')).codes)
X = ret.to_numpy()

X = torch.from_numpy(X).float()
y = torch.from_numpy(y).reshape(-1,1).long()
for sample in X:
    torch.save(sample.clone(),f'data_pt/{idx}.pt')
    idx += 1

In [ ]:
## Save y labels for all files
import pandas as pd
import numpy as np
ret = pd.DataFrame()
idxs = list(range(0,32))
df = load_psd(idxs[0])
y = df['label']

ret = pd.concat([ret,y])
for i in idxs[1:]:
    df = load_psd(i)
    y = df['label']

    ret = pd.concat([ret,y])

ret = ret.reset_index(drop=True)
ret = ret[ret[0] != 'X']
y = np.array(pd.Categorical(ret.pop(0)).codes)
y = torch.from_numpy(y).reshape(-1,1).long()
y = one_hot(y,num_classes=3).reshape(-1,3).float()
torch.save(y,'data_pt/labels.pt')

In [ ]:
import os
import pandas as pd
from torch.utils.data import Dataset,DataLoader

class EEGDataset(Dataset):
    def __init__(self,dir,labels):
        self.labels = torch.load(f'{labels}')
        self.dir = dir

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        path = os.path.join(self.dir, str(idx)+".pt")
        print(path)
        X = torch.load(path).reshape(-1,5000)
        y = self.labels[idx]

        return (X,y)
train_dataloader = DataLoader(EEGDataset(dir='pt/train',labels='pt/y_train.pt'), batch_size=500, shuffle=False)
test_dataloader = DataLoader(EEGDataset(dir='pt/test',labels='pt/y_test.pt'), batch_size=500, shuffle=False)

In [ ]:
from sklearn.model_selection import train_test_split
import torch
y = torch.load('pt/labels.pt')
train_idx,test_idx,_,_ = train_test_split(range(276399),y,shuffle=True,stratify=y,test_size=.2,random_state=0)

In [ ]:
os.system('mkdir pt/train')
os.system('mkdir pt/test')

for i,idx in enumerate(train_idx):
    os.system(f'mv pt/{idx}.pt pt/train/{i}.pt')
for i,idx in enumerate(test_idx):
    os.system(f'mv pt/{idx}.pt pt/test/{i}.pt')
torch.save(y[train_idx],'pt/y_train.pt')
torch.save(y[test_idx],'pt/y_test.pt')